In [82]:
from keras import backend as K
import time
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np_utils
%matplotlib inline
from keras.utils import multi_gpu_model
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc

In [33]:
import sys
import os
sys.path.append(os.getcwd())
from importlib import reload
import lib
reload(lib)
from lib import *

from oclr import OneCycleLR, LRFinder

<module 'lib' from '/home/ec2-user/SageMaker/ML_hackathon_2019/lib.py'>

In [4]:
df_train = pd.read_csv("asin_classification/train.csv")
df_test = pd.read_csv("asin_classification/test.csv")

In [5]:
df_train.shape
df_test.shape

(2315162, 4)

(100000, 4)

In [6]:
df_train['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text']))
df_train['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_train['text_encoded']))

df_test['text'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text']))
df_test['text_encoded'] = Parallel(n_jobs=16, backend="loky")(delayed(ast.literal_eval)(x) for x in tqdm(df_test['text_encoded']))


In [ ]:
start = time()
glove = api.load("glove-twitter-50") 
print("total = ",(time()-start))

In [34]:


ptr_glove_50 = PreTrainedEmbeddingsTransformer(glove,size=50)
ptr_glove_50.fit()

df_test['glove_encoded-50'] = ptr_glove_50.transform(df_test['text'].values)
df_train['glove_encoded-50'] = ptr_glove_50.transform(df_train['text'].values)


Fasttext Transforms start at: 2019-06-13 20:18:34.791308
Number of Unique Test Tokens for Fasttext transform 57550



Fasttext Transforms done at: 2019-06-13 20:18:39.466797
Fasttext Transforms start at: 2019-06-13 20:18:39.512225
Number of Unique Test Tokens for Fasttext transform 283052



Fasttext Transforms done at: 2019-06-13 20:19:03.966135


In [ ]:
start = time()
fasttext = api.load("fasttext-wiki-news-subwords-300") 
print("total = ",(time()-start))

ptr = PreTrainedEmbeddingsTransformer(fasttext,size=300)
ptr.fit()

df_train['fasttext_encoded'] = ptr.transform(df_train['text'].values)
df_test['fasttext_encoded'] = ptr.transform(df_test['text'].values)

In [35]:
batch_size = 512
epochs = 5


text_max_features = 50000
text_maxlen = 100
text_embedding_dims = 20

enc_maxlen = 100
enc_embedding_dims = 50

In [ ]:


# uniques[np.argmax(y_code,1)]
# uniques[y_code.argmax(1)]

In [41]:



X_enc,X_text,y = df_train['glove_encoded-50'].values,df_train['text_encoded'].values,df_train['target'].values
X_enc = sequence.pad_sequences(X_enc, maxlen=enc_maxlen)
X_text = sequence.pad_sequences(X_text, maxlen=text_maxlen)

uniques, coded_id = np.unique(y, return_inverse=True)
y = np_utils.to_categorical(coded_id, len(uniques))

x_train_enc, x_test_enc,x_train_text, x_test_text, y_train, y_test = train_test_split(X_enc,X_text, y, test_size=0.2, random_state=42)

x_train_enc = x_train_enc.reshape((-1,100,enc_embedding_dims,1))
x_test_enc = x_test_enc.reshape((-1,100,enc_embedding_dims,1))

datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,)
datagen.fit(x_train_enc)
train_iterator = datagen.flow(x_train_enc, y_train, batch_size=len(x_train_enc),shuffle=True)
validation_iterator = datagen.flow(x_test_enc, y_test, batch_size=len(x_test_enc),shuffle=True)

x_train_enc = train_iterator.next()[0]
x_test_enc = validation_iterator.next()[0]

x_train_enc = x_train_enc.reshape((-1,100,enc_embedding_dims))
x_test_enc = x_test_enc.reshape((-1,100,enc_embedding_dims))




# Helper

In [79]:
def intermediate_joiner(intermediates):
    x = concatenate(intermediates)
    x = transition_layer(x, n_kernels=128,dropout=0)
    x1 = conv_layer(x,n_kernels=32,kernel_size=3,padding='same',dilation_rate=2)
    x2 = conv_layer(x,n_kernels=128,kernel_size=3,padding='same')
    x = concatenate([x,x1,x2])
    
    x = transition_layer(x, n_kernels=64,dropout=0)
    x = conv_layer(x,n_kernels=len(uniques),kernel_size=3,padding='same',dropout=0)
    print("Before FC Intermediate =",K.int_shape(x))
    out = GlobalAveragePooling1D()(x)
    
    return out


In [59]:
def aux_output_fcnn(inputs):
    x = Activation("softmax")(inputs)
    return x

In [78]:
def word_cnn(inputs):
    x = inputs
    filters = 128
    kernel_size = 3
    x = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)

    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])

    xp = x
    x = transition_layer(x, n_kernels=64,dropout=0)
    x = conv_layer(x, n_kernels=128,kernel_size=3,dilation_rate=2,padding='same')
    x = concatenate([x,xp])
    x = transition_layer(x, n_kernels=32,dropout=0)
    x = conv_layer(x,n_kernels=len(uniques),kernel_size=3,dilation_rate=2,padding='same',dropout=0)
    intermidate = x
    # we use max pooling:
    x = GlobalMaxPooling1D()(x)
    
    print("W1 Intermediate =",K.int_shape(intermidate))
    return intermidate, aux_output_fcnn(x)

In [77]:
def pretrained_embedding_cnn():
    main_input = Input(shape=(enc_maxlen,enc_embedding_dims), dtype='float32')
    x = main_input
    x = conv_layer(x,n_kernels=128,kernel_size=3,padding='valid')
    
    x1 = MaxPooling1D(pool_size=2)(x)
    x2 = AveragePooling1D(pool_size=2)(x)
    x = concatenate([x1,x2])
    
    xp = x
    x = transition_layer(x, n_kernels=64,dropout=0)
    x = conv_layer(x,n_kernels=128,kernel_size=3,padding='same')
    x = transition_layer(x, n_kernels=64,dropout=0)


    x = concatenate([x,xp])
    x = transition_layer(x, n_kernels=32,dropout=0)
    x = conv_layer(x,n_kernels=len(uniques),kernel_size=3,dilation_rate=2,padding='same',dropout=0)
    
    

    intermidate = x
    print("Pretrained Intermediate =",K.int_shape(intermidate))
    x = GlobalAveragePooling1D()(x)
    return main_input,intermidate, aux_output_fcnn(x)

In [61]:
def final_layer(flattened):
    
    x = flattened
    print("Final Layer incoming = ",K.int_shape(x))

    # We add a vanilla hidden layer:
    x = Activation("softmax")(x)
    
    return x

# Ensemble

In [80]:
word_input = Input(shape=(text_maxlen,), dtype='int32')
x = Embedding(text_max_features,
            text_embedding_dims,
            input_length=text_maxlen)(word_input)

w1_intermidate, w1_output = word_cnn(x)

p_inputs,p_intermidate, p_output = pretrained_embedding_cnn()


intermediates = [w1_intermidate,p_intermidate]
x = intermediate_joiner(intermediates)

op = final_layer(x)

outputs = [w1_output,p_output,op]
inputs = [word_input,p_inputs]

model = Model(inputs=inputs, outputs=outputs)

optimizer = Adam(lr=0.0001,)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'],loss_weights=[0.2,0.2,1.0])

print("Params = ",model.count_params())
# 13331414
# 3562108
# 3227580

W1 Intermediate = (None, 49, 1316)
Pretrained Intermediate = (None, 49, 1316)
Before FC Intermediate = (None, 49, 1316)
Final Layer incoming =  (None, 1316)
Params =  2086460


In [81]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 100, 50)      0                                            
__________________________________________________________________________________________________
conv1d_127 (Conv1D)             (None, 98, 128)      19328       input_19[0][0]                   
__________________________________________________________________________________________________
batch_normalization_117 (BatchN (None, 98, 128)      512         conv1d_127[0][0]                 
__________________________________________________________________________________________________
activation_147 (Activation)     (None, 98, 128)      0           batch_normalization_117[0][0]    
__________________________________________________________________________________________________
dropout_12

In [ ]:
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer=optimizer,
              metrics=['acc'],loss_weights=[0.2,0.2,1.0])

parallel_model.fit([x_train_text,x_train_enc], [y_train,y_train,y_train],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test_text,x_test_enc], [y_test,y_test,y_test]),
         shuffle=True)

In [ ]:
model.fit([x_train_text,x_train_enc], [y_train,y_train,y_train],
          batch_size=batch_size,
          epochs=epochs,
          validation_data=([x_test_text,x_test_enc], [y_test,y_test,y_test]),
         shuffle=True)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1159686 samples, validate on 289922 samples
Epoch 1/5
